In [ ]:
# calculate memory %
memorypct = udf(lambda x: x[0]/x[1], FloatType())
dfeng = dfeng.withColumn('memory_pct', memorypct(struct('memory_in_use_mb','memory_total_mb')))

# filter delts=T
dfeng = dfeng.filter(col('delta')==True)

In [ ]:
# Get avg and std for temp and memory% per switch
df0.groupby('mac','name','re_ix').agg(avg('temp_celsius').alias('avg_temp'), stddev('temp_celsius').alias('std_temp'), \
                                      avg('memory_pct').alias('avg_memory'), stddev('memory_pct').alias('std_memory'))

# Get the upper limit for temp and memory% per switch, overseeding this limit will be AD (yellow alarm)
upper = udf(lambda x: x[0]+3*x[1], FloatType())
df0 = df0.withColumn('upper_temp', upper('avg_temp', 'std_temp')) \
            .withColumn('upper_memory', upper('avg_memory', 'std_memory'))


In [1]:
# smoothing data
df0 = df0.select('mac','name','re_ix','temp_celsius','avg_temp','std_temp','memory_pct','avg_memory','std_memory','cpu_user','power_pct')
pdf = df0.toPandas()
temp_l = smooth(pdf['temp_celsius'].tolist())
pdf['temp_s'] = pd.DataFrame(temp_l)
memory_l = smooth(pdf['memory_pct'].tolist())
pdf['memory_s'] = pd.DataFrame(memory_l)
cpuuser_l = smooth(pdf['cpu_user'].tolist())
pdf['cpuuser_s'] = pd.DataFrame(cpuuser_l)
power_l = smooth(pdf['power_pct'].tolist())
pdf['power_s'] = pd.DataFrame(power_l)

# convert back to spark dataframe
df0 = spark_session.createDataFrame(pdf)


NameError: name 'df0' is not defined

In [ ]:
# Yellow AD for new data
yellow = udf(lambda x: 1 if x[0]>x[1] else 0, IntegerType())
df0 = df0.withColumn('yellow_temp', yellow('temp_celsius', 'upper_temp')) \
            .withColumn('yellow_memory', yellow('memory_pct', 'upper_memory'))

# Red Alarm for new data 
red = udf(lambda x: 1 if x>=0.9 else 0, IntegerType())
redtemp = udf(lambda x: 1 if x>=70 else 0, IntergerType())
df0 = df0.withColumn('red_temp', redtemp('temp_celsius')) \
            .withColumn('red_memory', red('pct_memory')) \
            .withColumn('red_user', red('cpu_user')) \
            .withColumn('red_power', red('power_pct'))

